In [2]:
library(Seurat)

library(dplyr)
library(glue)
library(ggplot2)
library(patchwork)
library(cowplot)
library(stringr)
library(tidyr)
library("dplyr")
library(velocyto.R)
library(SeuratWrappers)
load("6_18.RData")

Attaching SeuratObject


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'glue'


The following object is masked from 'package:dplyr':

    collapse



Attaching package: 'cowplot'


The following object is masked from 'package:patchwork':

    align_plots


Loading required package: Matrix


Attaching package: 'Matrix'


The following objects are masked from 'package:tidyr':

    expand, pack, unpack




In [3]:
?ScaleData

ScaleData {Seurat},R Documentation
object,An object
...,Arguments passed to other methods
features,Vector of features names to scale/center. Default is variable features.
vars.to.regress,"Variables to regress out (previously latent.vars in RegressOut). For example, nUMI, or percent.mito."
latent.data,"Extra data to regress out, should be cells x latent data"
split.by,Name of variable in object metadata or a vector or factor defining grouping of cells. See argument f in split for more details
model.use,"Use a linear model or generalized linear model (poisson, negative binomial) for the regression. Options are 'linear' (default), 'poisson', and 'negbinom'"
use.umi,"Regress on UMI count data. Default is FALSE for linear modeling, but automatically set to TRUE if model.use is 'negbinom' or 'poisson'"
do.scale,Whether to scale the data.
do.center,Whether to center the data.


In [5]:
library(SeuratDisk)
a=as.loom(merged_obj, filename = "merged_obj.loom", verbose = T,overwrite=T)
a$close_all()

Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Saving data from RNA as /matrix

Adding slot counts for assay RNA

Adding layer counts

Adding col attribute CellID

Adding col attribute orig.ident

Adding col attribute nCount_RNA

Adding col attribute nFeature_RNA

Adding col attribute nCount_ADT

Adding col attribute nFeature_ADT

Adding col attribute percent.mt

Adding col attribute RNA.weight

Adding col attribute ADT.weight

Adding col attribute wsnn_res.1

Adding col attribute seurat_clusters

Adding col attribute Blood_pred

Adding col attribute BM_pred

Adding col attribute Bone_pred

Adding col attribute id

Adding row attribute Gene



In [3]:
l1 <- ReadVelocity(file = "mWT/velocyto/mWT.loom")
l2 <- ReadVelocity(file = "mNFIXko/velocyto/mNFIXko.loom")
s1 = as.Seurat(x=l1,orig.ident="mWT")
s2 = as.Seurat(x=l2,orig.ident="mNFIXko")

Warning message:
“Non-unique features (rownames) present in the input matrix, making unique”
Warning message:
“Non-unique features (rownames) present in the input matrix, making unique”


In [5]:
a=colnames(s1)
b=str_replace(a,":","_")
b=str_replace(b,"x","-1")
s1$orig.ident <- "mWT"
s1 = RenameCells(s1,new.names=b)

a=colnames(s2)
b=str_replace(a,":","_")
b=str_replace(b,"x","-1")
s2$orig.ident <- "mNFIXko"
s2 = RenameCells(s2,new.names=b)


In [19]:
loom_obj = merge(s1, y =s2)
loom_obj = subset(loom_obj, cells = colnames(merged_obj))
genes_in_use <- rownames(GetAssayData(merged_obj, assay = "RNA", slot = "data"))

spliced <- GetAssayData(loom_obj, assay = "spliced") %>% .[rownames(.) %in% genes_in_use,] %>% CreateAssayObject()
unspliced <- GetAssayData(loom_obj, assay = "unspliced") %>% .[rownames(.) %in% genes_in_use,] %>% CreateAssayObject()
ambiguous <- GetAssayData(loom_obj, assay = "ambiguous") %>% .[rownames(.) %in% genes_in_use,] %>% CreateAssayObject()


merged_obj[["spliced"]] <- spliced
merged_obj[["unspliced"]] <- unspliced
merged_obj[["ambiguous"]] <- ambiguous


# merged_obj[['spliced']] = loom_obj[['spliced']]
# merged_obj[['unspliced']] = loom_obj[['unspliced']]
# merged_obj[['ambiguous']] = loom_obj[['ambiguous']]
merged_obj

An object of class Seurat 
124220 features across 13949 samples within 5 assays 
Active assay: RNA (31053 features, 2000 variable features)
 4 other assays present: ADT, spliced, unspliced, ambiguous
 5 dimensional reductions calculated: pca, apca, wnn.umap, rna.umap, adt.umap

In [36]:
dim(merged_obj[['RNA']])

[1] 31053 13949

In [11]:
save.image("7_29_velocity.RData")

In [30]:
head(merged_obj@assays$RNA@data)

   [[ suppressing 13949 column names ‘mWT_AAACCCAAGTGATCGG-1’, ‘mWT_AAACCCACAAACCATC-1’, ‘mWT_AAACCCACACTGTCCT-1’ ... ]]



6 x 13949 sparse Matrix of class "dgCMatrix"
                                                                               
Xkr4    . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Gm1992  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Gm37381 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Rp1     . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Sox17   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Gm37323 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
                                                                               
Xkr4    . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Gm1992  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Gm37381 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Rp1     . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [31]:
head(merged_obj@assays$spliced@data)

   [[ suppressing 13949 column names ‘mWT_AAACCCAAGTGATCGG-1’, ‘mWT_AAACCCACAAACCATC-1’, ‘mWT_AAACCCACACTGTCCT-1’ ... ]]



6 x 13949 sparse Matrix of class "dgCMatrix"
                                                                               
Xkr4    . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Gm37381 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Rp1     . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Sox17   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Gm37323 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Mrpl15  . . . . 2 3 . 2 1 . 1 2 . . 2 1 1 2 . . . . . 2 2 . . . 1 . 1 . . 2 . 1
                                                                               
Xkr4    . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Gm37381 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Rp1     . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Sox17   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [34]:
31053  *5

[1] 155265

In [7]:
merged_obj

An object of class Seurat 
31061 features across 13949 samples within 2 assays 
Active assay: RNA (31053 features, 2000 variable features)
 1 other assay present: ADT
 5 dimensional reductions calculated: pca, apca, wnn.umap, rna.umap, adt.umap

In [21]:
library(SeuratDisk)
DefaultAssay(merged_obj) <- "RNA"
SaveH5Seurat(merged_obj, filename = "mouseBM_with_velo.h5Seurat",overwrite =T)
Convert("mouseBM_with_velo.h5Seurat", dest = "h5ad",overwrite =T)

Warning message:
“Overwriting previous file mouseBM_with_velo.h5Seurat”
Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

Adding scale.data for RNA

Adding variable features for RNA

Adding feature-level metadata for RNA

Adding counts for ADT

Adding data for ADT

Adding scale.data for ADT

Adding variable features for ADT

No feature-level metadata found for ADT

Adding counts for spliced

Adding data for spliced

No variable features found for spliced

No feature-level metadata found for spliced

Adding counts for unspliced

Adding data for unspliced

No variable features found for unspliced

No feature-level metadata found for unspliced

Adding counts for ambiguous

Adding data for ambiguous

No variable features found for ambiguous

No feature-level metadata found for ambiguous

Adding cell embeddings for pca

Adding loadings for pca

No projected loadings for pca

Adding standard deviations for pca

No JackStraw data for pca

Adding cell e

In [43]:
# https://github.com/satijalab/seurat/issues/3654

In [42]:
write.csv(x = t(as.matrix(merged_obj@assays$spliced@counts)), file = 'spliced.csv')
write.csv(x = t(as.matrix(merged_obj@assays$unspliced@counts)), file = 'unspliced.csv')

In [1]:
merged_obj

ERROR: Error in eval(expr, envir, enclos): object 'merged_obj' not found


In [40]:
a=as.loom(merged_obj, filename = "mouseBM_with_velo.loom", verbose = T,overwrite=T)
a$close_all()

Warning message:
“Overwriting previous file mouseBM_with_velo.loom”
Saving data from RNA as /matrix

Adding slot counts for assay RNA

Adding layer counts

Adding col attribute CellID

Adding col attribute orig.ident

Adding col attribute nCount_RNA

Adding col attribute nFeature_RNA

Adding col attribute nCount_ADT

Adding col attribute nFeature_ADT

Adding col attribute percent.mt

Adding col attribute RNA.weight

Adding col attribute ADT.weight

Adding col attribute wsnn_res.1

Adding col attribute seurat_clusters

Adding col attribute Blood_pred

Adding col attribute BM_pred

Adding col attribute Bone_pred

Adding col attribute id

Adding col attribute nCount_spliced

Adding col attribute nFeature_spliced

Adding col attribute nCount_unspliced

Adding col attribute nFeature_unspliced

Adding col attribute nCount_ambiguous

Adding col attribute nFeature_ambiguous

Adding row attribute Gene

